In [1]:
from dataset import CustomDataset
from runner.model import TimmModel
from config import CFG

from utils_.set_seed import seed_everything
from utils_.set_path import *


/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import timm

import warnings
warnings.filterwarnings(action='ignore') 

import wandb

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [4]:
df = pd.read_csv(TRAIN_CSV_PATH)
df['filename'] = df['filename'].apply(lambda x : os.path.join(TRAIN_IMG_FOLDER_PATH,x))

In [5]:
df.head()

,id,filename,label
0,0,/opt/ml/open/Data/Training_whole/PNG/피스_44.png,피스
1,1,/opt/ml/open/Data/Training_whole/PNG/피스_23.png,피스
2,2,/opt/ml/open/Data/Training_whole/PNG/피스_33.png,피스
3,3,/opt/ml/open/Data/Training_whole/PNG/피스_29.png,피스
4,4,/opt/ml/open/Data/Training_whole/PNG/피스_27.png,피스


In [6]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

In [7]:
indexes = val['label'].value_counts().index
val['label'].value_counts()

훼손         422
오염         179
걸레받이수정      92
꼬임          63
터짐          49
곰팡이         43
오타공         43
몰딩수정        39
면불량         30
석고수정        17
들뜸          16
피스          15
창틀,문틀수정      8
울음           7
이음부불량        5
녹오염          4
가구수정         4
반점           1
틈새과다         1
Name: label, dtype: int64

In [8]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [9]:
val['label'].value_counts().values

array([422, 179,  92,  63,  49,  43,  43,  39,  30,  17,  16,  15,   8,
         7,   5,   4,   4,   1,   1])

# Dataset

In [10]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [11]:
val_dataset = CustomDataset(val['filename'].values, val['label'].values, transforms = False, CFG=CFG)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

# Model load

In [12]:
import timm
from pprint import pprint
model_names = timm.list_models('vit_base_patch16*')
(model_names)

['vit_base_patch16_18x2_224',
 'vit_base_patch16_224',
 'vit_base_patch16_224_dino',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_224_sam',
 'vit_base_patch16_384',
 'vit_base_patch16_plus_240',
 'vit_base_patch16_rpn_224']

In [13]:
model = TimmModel('efficientnet_b3', num_classes = 19, pretrained= True)

In [14]:
model.state_dict()

OrderedDict([('model.conv_stem.weight',
              tensor([[[[ 6.4226e-02,  5.8936e-02, -5.4534e-02],
                        [ 1.1693e-01, -2.1102e-01, -4.1341e-01],
                        [ 1.0670e-01,  1.5955e-02, -1.3696e-01]],
              
                       [[ 6.9534e-02,  1.0569e-01, -2.2195e-02],
                        [ 7.8271e-02, -2.5755e-01, -4.6491e-01],
                        [ 6.6158e-02,  1.0488e-01, -1.3329e-01]],
              
                       [[ 8.3416e-02,  1.2411e-01,  1.3346e-03],
                        [ 7.3505e-02, -1.9720e-01, -3.8826e-01],
                        [ 8.5462e-02,  1.2862e-02, -1.2831e-01]]],
              
              
                      [[[-2.2875e-02, -3.2713e-02, -4.2322e-02],
                        [-5.7387e-02,  1.7484e-01,  1.0626e-01],
                        [ 5.4029e-03,  6.4526e-02,  1.0856e-01]],
              
                       [[-2.7656e-01, -4.7466e-01, -5.5577e-01],
                        [-2.4555e-0

In [15]:
new = dict()
tmp = (torch.load('/opt/ml/cv16-Wallpaper-defect_classification/models/[efficientnet_b3]_[score0.8321]_[loss0.6985]_1e-3.pt'))
model = nn.DataParallel(model) # 해줘야 .module 이 사라지구나
model.load_state_dict(tmp)
model.eval()
model.to(device)

DataParallel(
  (module): TimmModel(
    (model): EfficientNet(
      (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNormAct2d(
        40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): SiLU(inplace=True)
      )
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (bn1): BatchNormAct2d(
              40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): SiLU(inplace=True)
            )
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
              (gate): Sigmoi

In [16]:
# for key, value in tmp.items():
#     if 'backbone' in key:
#         new[key.replace('backbone', 'model')] = value 
#     # else:
#     #     new[key.replace('fc', 'module.fc')] = value
# model.load_state_dict(new)
# model.eval()
# model.to(device)
# new

# 예측 결과 해석

In [17]:
targets_img = None
preds, targets = torch.Tensor().to(device), torch.Tensor().to(device)
prediction = []
with torch.no_grad():
    for X, y in tqdm(val_loader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        pred = pred.argmax(dim=1)
        indices = (pred != y).nonzero().squeeze()

        if indices.nelement() != 0:
            target_img = X[indices]
            if target_img.dim() == 3:
                target_img = target_img.unsqueeze(dim=0)
            
            if targets_img is None:
                targets_img = target_img
            else:
                targets_img = torch.cat((targets_img, target_img), dim=0)

            ps = pred[indices]
            ys = y[indices]


            if ys.dim() == 0: # 배치에사 하나만 잘 못 판단하면 이 코드 넣어주어야함
                # print(ps, ys)
                # print(ys.dim())
                # break
                ys = ys.unsqueeze(dim=0)
                ps = ps.unsqueeze(dim=0)

            preds = torch.cat((preds, ps))
            targets = torch.cat((targets, ys))
            print(preds, targets)

tensor([18.,  6., 11.], device='cuda:0') tensor([ 6., 15., 18.], device='cuda:0')
tensor([18.,  6., 11., 10.,  1.], device='cuda:0') tensor([ 6., 15., 18., 18.,  2.], device='cuda:0')
tensor([18.,  6., 11., 10.,  1., 18., 17., 15.], device='cuda:0') tensor([ 6., 15., 18., 18.,  2.,  3.,  4., 10.], device='cuda:0')
tensor([18.,  6., 11., 10.,  1., 18., 17., 15., 18., 10.,  6., 10.,  5.,  0.],
       device='cuda:0') tensor([ 6., 15., 18., 18.,  2.,  3.,  4., 10., 10., 18., 10.,  4., 14., 10.],
       device='cuda:0')
tensor([18.,  6., 11., 10.,  1., 18., 17., 15., 18., 10.,  6., 10.,  5.,  0.,
        10.], device='cuda:0') tensor([ 6., 15., 18., 18.,  2.,  3.,  4., 10., 10., 18., 10.,  4., 14., 10.,
        15.], device='cuda:0')
tensor([18.,  6., 11., 10.,  1., 18., 17., 15., 18., 10.,  6., 10.,  5.,  0.,
        10., 13., 10.], device='cuda:0') tensor([ 6., 15., 18., 18.,  2.,  3.,  4., 10., 10., 18., 10.,  4., 14., 10.,
        15., 12., 18.], device='cuda:0')
tensor([18.,  6., 11.,

# 총 틀린 prdes, targets 개수 

In [18]:
worng_preds = pd.Series(list(map(int, preds.cpu()))).value_counts()
worng_targets = pd.Series(list(map(int, targets.cpu()))).value_counts()


In [19]:
worng_targets

18    8
10    8
15    2
6     2
4     2
17    1
14    1
12    1
3     1
2     1
1     1
dtype: int64

In [20]:
worng_preds

18    8
10    7
15    3
6     3
11    2
17    1
13    1
5     1
1     1
0     1
dtype: int64

In [25]:
sorted(pd.Series(list(map(int,targets))).unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [21]:
total_cnt = val['label'].value_counts()
indexes, val['label'].value_counts()

(Index(['훼손', '오염', '걸레받이수정', '꼬임', '터짐', '곰팡이', '오타공', '몰딩수정', '면불량', '석고수정',
        '들뜸', '피스', '창틀,문틀수정', '울음', '이음부불량', '녹오염', '가구수정', '반점', '틈새과다'],
       dtype='object'),
 18    422
 10    179
 1      92
 3      63
 15     49
 2      43
 11     43
 7      39
 6      30
 9      17
 5      16
 17     15
 14      8
 12      7
 13      5
 4       4
 0       4
 8       1
 16      1
 Name: label, dtype: int64)

In [22]:
from collections import defaultdict

In [23]:
worng_pred = defaultdict(int)
worng_target = defaultdict(int)
for key in list(worng_preds.keys()):
    worng_pred[key] = worng_preds[key] / total_cnt[key] # len(dataloader)로 바꾸자
for key in list(worng_targets.keys()):
    worng_target[key] = worng_targets[key] / total_cnt[key]    

In [24]:
worng_pred, worng_target
# 잘못 예측한 preds, 잘못 예측한 true label

(defaultdict(int,
             {18: 0.018957345971563982,
              10: 0.03910614525139665,
              15: 0.061224489795918366,
              6: 0.1,
              11: 0.046511627906976744,
              17: 0.06666666666666667,
              13: 0.2,
              5: 0.0625,
              1: 0.010869565217391304,
              0: 0.25}),
 defaultdict(int,
             {18: 0.018957345971563982,
              10: 0.0446927374301676,
              15: 0.04081632653061224,
              6: 0.06666666666666667,
              4: 0.5,
              17: 0.06666666666666667,
              14: 0.125,
              12: 0.14285714285714285,
              3: 0.015873015873015872,
              2: 0.023255813953488372,
              1: 0.010869565217391304}))

In [34]:
misprediction_count = {
    (18, 10) : 0,
    (18, 1) : 0,
    (1, 0) : 0,
    (1, 2) : 0,
    (2, 0) : 0,
    (2, 1) : 0
}
misprediction_count = defaultdict

misprediction_count = { (i, 17) : 0 for i in range(19)}
print(misprediction_count)

{(0, 17): 0, (1, 17): 0, (2, 17): 0, (3, 17): 0, (4, 17): 0, (5, 17): 0, (6, 17): 0, (7, 17): 0, (8, 17): 0, (9, 17): 0, (10, 17): 0, (11, 17): 0, (12, 17): 0, (13, 17): 0, (14, 17): 0, (15, 17): 0, (16, 17): 0, (17, 17): 0, (18, 17): 0}


In [35]:
for pred, label in tqdm(zip(preds, targets)):
    if (pred.item(), label.item()) in misprediction_count.keys():
        misprediction_count[(pred.item(), label.item())] += 1

In [36]:
misprediction_count

{(0, 17): 0,
 (1, 17): 0,
 (2, 17): 0,
 (3, 17): 0,
 (4, 17): 0,
 (5, 17): 0,
 (6, 17): 0,
 (7, 17): 0,
 (8, 17): 0,
 (9, 17): 0,
 (10, 17): 0,
 (11, 17): 0,
 (12, 17): 0,
 (13, 17): 0,
 (14, 17): 0,
 (15, 17): 0,
 (16, 17): 0,
 (17, 17): 0,
 (18, 17): 1}